In [1]:
from ipynb.fs.full.functions import import_data
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import optimizers, regularizers
import numpy as np
import keras.utils
import matplotlib.pyplot as plt
import pandas as pd

import talos as ta


Using TensorFlow backend.
C:\Users\auria\Anaconda3\envs\keras_environment\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
C:\Users\auria\Anaconda3\envs\keras_environment\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Load data

In [2]:
#X train also contains cross validation set. model.fit will split it between training and cross validation
X, X_train, X_test, y_train, y_test = import_data('Database_2.csv')
input_size = X_train.shape[1]

### Import parameters from layers tuning

In [3]:
first_layer = 512
second_layer = 128
batch_size = 32
epochs = 120

### Create model

In [4]:
def stock_picker_model(x_train, y_train, x_val, y_val, params):
    #Build model
    model = Sequential()
    model.add(Dense(first_layer, activation='relu', input_dim=input_size,kernel_regularizer=regularizers.l2(params['l2_reg']), ))
    model.add(Dense(second_layer, activation='relu', kernel_regularizer=regularizers.l2(params['l2_reg'])))

    model.add(Dense(1, activation='sigmoid'))
    
    #compile model
    optimizer = optimizers.Adam(lr=params['learning_rate'], beta_1 = 0.9, beta_2=0.999)
    model.compile(optimizer=optimizer,
                  loss = 'binary_crossentropy',
                  metrics=['accuracy'])
    
    #fit
    out = model.fit(x_train, y_train, 
                        validation_data=[x_val, y_val],
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0)
    
    return out, model

### Build parameters testing dictionnary

In [5]:
p = {'l2_reg' : [0.001, 0.01, 0.1],
    'learning_rate' : [0.01, 0.02, 0.04, 0.08, 0.16, 0.32, 0.64, 1.28]}

### Run grid search

In [6]:
t = ta.Scan(x=X_train,
            y=y_train,
            model=stock_picker_model,
            params=p,
            dataset_name='lr_and_regularization_report',
            experiment_no='1')
            
            

  0%|                                                                                           | 0/24 [00:00<?, ?it/s]

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [10:38<00:00, 26.66s/it]
